# Lesson 4A: Support Vector Machines TheorySVMs find the optimal decision boundary by maximizing the margin between classes.

## IntroductionImagine you're drawing a path between two groups of obstacles. You want the widest possible path - maximizing the safety margin on both sides. That's what SVM does mathematically.

## Table of Contents1. What is SVM?2. Maximum margin concept3. Support vectors4. The kernel trick5. Soft margins6. SVM from scratch

In [ ]:
import numpy as npimport matplotlib.pyplot as pltfrom sklearn.datasets import make_classification, load_breast_cancerfrom sklearn.model_selection import train_test_splitfrom sklearn.metrics import accuracy_score, classification_reportnp.random.seed(42)print('✅ Libraries loaded')

## Maximum MarginSVM finds the hyperplane that maximizes distance to nearest points of both classes.**Math:** Maximize margin = 2/||w|| subject to: y_i(w·x_i + b) ≥ 1**Support Vectors:** Points closest to the boundary - these define the margin.

In [ ]:
# Visualize conceptX, y = make_classification(n_samples=100, n_features=2, n_redundant=0, n_clusters_per_class=1, random_state=42)plt.figure(figsize=(10, 6))plt.scatter(X[y==0, 0], X[y==0, 1], label='Class 0', alpha=0.7)plt.scatter(X[y==1, 0], X[y==1, 1], label='Class 1', alpha=0.7)plt.xlabel('Feature 1')plt.ylabel('Feature 2')plt.title('SVM Goal: Find Widest Margin Between Classes', fontweight='bold')plt.legend()plt.grid(alpha=0.3)plt.show()print('✅ SVM maximizes the margin!')

## The Kernel TrickFor non-linear data, map to higher dimensions:- **Linear:** K(x, x') = x·x'- **Polynomial:** K(x, x') = (x·x' + c)^d- **RBF (Gaussian):** K(x, x') = exp(-γ||x-x'||²)Most popular: RBF kernel - creates smooth, non-linear boundaries.

## Simple SVM Implementation

In [ ]:
class SimpleSVM:    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):        self.lr = learning_rate        self.lambda_param = lambda_param        self.n_iters = n_iters        self.w = None        self.b = None    def fit(self, X, y):        n_samples, n_features = X.shape        y_ = np.where(y <= 0, -1, 1)  # Convert to -1, 1        self.w = np.zeros(n_features)        self.b = 0        for _ in range(self.n_iters):            for idx, x_i in enumerate(X):                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1                if condition:                    self.w -= self.lr * (2 * self.lambda_param * self.w)                else:                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))                    self.b -= self.lr * y_[idx]    def predict(self, X):        return np.sign(np.dot(X, self.w) - self.b)print('✅ SimpleSVM class created!')

In [ ]:
# Test on breast cancerdata = load_breast_cancer()X, y = data.data, data.targetX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# NormalizeX_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)svm = SimpleSVM(n_iters=500)svm.fit(X_train, y_train)y_pred = svm.predict(X_test)y_pred = np.where(y_pred <= 0, 0, 1)print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')print('✅ SVM from scratch works!')

## Conclusion**SVMs are powerful for:**- High-dimensional data- Clear margin of separation- Memory efficient (only support vectors matter)**Next:** Lesson 4B - Scikit-learn SVMs with different kernels!